Lambda School Data Science, Unit 2: Predictive Modeling

# Kaggle Challenge, Module 4

## Catch up, if needed
- [Review requirements for your portfolio project](https://lambdaschool.github.io/ds/unit2/portfolio-project/ds6), then choose your dataset, and [submit this form](https://forms.gle/nyWURUg65x1UTRNV9), due yesterday at 3:59pm Pacific.
- Submit predictions to our Kaggle competition. (Go to our Kaggle InClass competition webpage. Use the blue **Submit Predictions** button to upload your CSV file.) The competition closes today at 3:59pm. Every student should make at least one submission that scores at least 60% accuracy (above the majority class baseline).

## Assignment
- [ ] Continue to participate in our Kaggle challenge. 
- [ ] Use scikit-learn for hyperparameter optimization with RandomizedSearchCV.
- [ ] Submit your final predictions to our Kaggle competition. Optionally, go to **My Submissions**, and _"you may select up to 1 submission to be used to count towards your final leaderboard score."_ The competition closes today at 3:59pm.
- [ ] Add comments and Markdown to your notebook. Clean up your code.
- [ ] Commit your notebook to your fork of the GitHub repo.

## Stretch Goals

### Reading
- Jake VanderPlas, [Python Data Science Handbook, Chapter 5.3](https://jakevdp.github.io/PythonDataScienceHandbook/05.03-hyperparameters-and-model-validation.html), Hyperparameters and Model Validation
- Jake VanderPlas, [Statistics for Hackers](https://speakerdeck.com/jakevdp/statistics-for-hackers?slide=107)
- Ron Zacharski, [A Programmer's Guide to Data Mining, Chapter 5](http://guidetodatamining.com/chapter5/), 10-fold cross validation
- Sebastian Raschka, [A Basic Pipeline and Grid Search Setup](https://github.com/rasbt/python-machine-learning-book/blob/master/code/bonus/svm_iris_pipeline_and_gridsearch.ipynb)
- Peter Worcester, [A Comparison of Grid Search and Randomized Search Using Scikit Learn](https://blog.usejournal.com/a-comparison-of-grid-search-and-randomized-search-using-scikit-learn-29823179bc85)

### Doing
- Try combining xgboost early stopping, cross-validation, & hyperparameter optimization, with [the "original" (non scikit-learn) xgboost API](https://xgboost.readthedocs.io/en/latest/python/python_api.html#xgboost.cv).
- In additon to `RandomizedSearchCV`, scikit-learn has [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Another library called scikit-optimize has [`BayesSearchCV`](https://scikit-optimize.github.io/notebooks/sklearn-gridsearchcv-replacement.html). Experiment with these alternatives.
- _[Introduction to Machine Learning with Python](http://shop.oreilly.com/product/0636920030515.do)_ discusses options for "Grid-Searching Which Model To Use" in Chapter 6:

> You can even go further in combining GridSearchCV and Pipeline: it is also possible to search over the actual steps being performed in the pipeline (say whether to use StandardScaler or MinMaxScaler). This leads to an even bigger search space and should be considered carefully. Trying all possible solutions is usually not a viable machine learning strategy. However, here is an example comparing a RandomForestClassifier and an SVC ...

The example is shown in [the accompanying notebook](https://github.com/amueller/introduction_to_ml_with_python/blob/master/06-algorithm-chains-and-pipelines.ipynb), code cells 35-37. Could you apply this concept to your own pipelines?

#### Try stacking multiple submissions!

Here's some code you can use:

```python
import pandas as pd

# Filenames of your submissions you want to ensemble
files = ['submission-01.csv', 'submission-02.csv', 'submission-03.csv']

target = 'status_group'
submissions = (pd.read_csv(file)[[target]] for file in files)
ensemble = pd.concat(submissions, axis='columns')
majority_vote = ensemble.mode(axis='columns')[0]

sample_submission = pd.read_csv('sample_submission.csv')
submission = sample_submission.copy()
submission[target] = majority_vote
submission.to_csv('my-ultimate-ensemble-submission.csv', index=False)
```

In [14]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split as tts
import category_encoders as ce
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from eli5.sklearn import PermutationImportance
import eli5
from scipy.stats import randint, uniform
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score

pd.set_option('display.max_columns', None)

In [2]:
# creates dataframes for each file

test0 = pd.read_csv('https://raw.githubusercontent.com/SeanAntosiak/DS-Unit-2-Regression-Classification/master/data/tanzania/test_features.csv')
train0 = pd.read_csv('https://raw.githubusercontent.com/SeanAntosiak/DS-Unit-2-Regression-Classification/master/data/tanzania/train_features.csv')
trainLabels = pd.read_csv('https://raw.githubusercontent.com/SeanAntosiak/DS-Unit-2-Regression-Classification/master/data/tanzania/train_labels.csv')
sample = pd.read_csv('https://raw.githubusercontent.com/SeanAntosiak/DS-Unit-2-Regression-Classification/master/data/tanzania/sample_submission.csv')

In [3]:
# performs a train test split to create train and validation data

Xtrain0, Xval0, ytrain0, yval0 = tts(train0, trainLabels['status_group'], train_size=0.85, test_size=0.15, random_state=8)

In [4]:
# creates a function to wrangle data

def wrangle(DF):
    
    # creates a copy of the input dataframe
    df = DF.copy()
                    
    # converts date_recorded to a year
    df['date_recorded'] = pd.to_datetime(df['date_recorded'], infer_datetime_format=True).dt.year
    
    # defines the columns which apear to have missing values input as 0
    zeroCols = ['amount_tsh', 'longitude', 'latitude', 'gps_height', 'construction_year']
    
     # replaces tiny latitude values with 0 to be replaced with nan in the next step 
    df['latitude']=df['latitude'].replace(-2e-8,0)
    
    # replaces missing 0 values with nan
    for col in zeroCols:
        df[col] = df[col].replace(0, np.nan)
        df[col+'_missing']=df[col].isnull();
       
        
    return(df);

In [5]:
# applies wrangle function to each dataframe

Xtrain = wrangle(Xtrain0)
Xval = wrangle(Xval0)
Xtest = wrangle(test0)

In [21]:
# creates a pipeline for a random forrest regressor then scores it with cross score validation

pipe0 = make_pipeline(
            ce.OrdinalEncoder(), 
            SimpleImputer(),
            MinMaxScaler(),     
            RandomForestClassifier()
)

In [22]:
# fits pipeline and scores it

pipe0.fit(Xtrain, ytrain0)

pipe0.score(Xval, yval0)

/opt/anaconda/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.8023569023569024

In [31]:
search = RandomizedSearchCV(
            pipe0,
            param_distributions = {
            'simpleimputer__strategy':['mean','median','most_frequent'],
            'randomforestclassifier__n_estimators': randint(10,100), # I am running into memory errors if this value is higher
            'randomforestclassifier__max_depth': [5, 10, 15, 20], 
            'randomforestclassifier__max_features': uniform(0, 1)
            },
        n_iter=5,
        cv=3,
        return_train_score=True,
        n_jobs=-1
)

search.fit(Xtrain, ytrain0);

In [32]:
pd.DataFrame(search.cv_results_).sort_values(by='rank_test_score')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_randomforestclassifier__max_depth,param_randomforestclassifier__max_features,param_randomforestclassifier__n_estimators,param_simpleimputer__strategy,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,mean_train_score,std_train_score
0,8.633264,0.043811,0.536534,0.001811,20,0.244742,61,median,"{'randomforestclassifier__max_depth': 20, 'ran...",0.801973,0.800368,0.805324,0.802555,0.002065,1,0.973410,0.974004,0.974719,0.974044,0.000535
4,16.747514,1.518296,0.371462,0.021300,20,0.598392,81,median,"{'randomforestclassifier__max_depth': 20, 'ran...",0.799002,0.799834,0.802234,0.800357,0.001370,2,0.965923,0.964675,0.965718,0.965439,0.000546
2,7.626476,0.163416,0.477811,0.003329,15,0.290425,50,median,"{'randomforestclassifier__max_depth': 15, 'ran...",0.797398,0.792942,0.800749,0.797029,0.003198,3,0.897947,0.890698,0.891480,0.893375,0.003249
1,27.773251,0.443891,0.559022,0.012275,15,0.786308,87,median,"{'randomforestclassifier__max_depth': 15, 'ran...",0.790209,0.789317,0.794212,0.791246,0.002129,4,0.890579,0.880507,0.883608,0.884898,0.004212
3,5.620033,0.018346,0.375247,0.011620,10,0.447509,31,median,"{'randomforestclassifier__max_depth': 10, 'ran...",0.764304,0.759194,0.762123,0.761874,0.002093,5,0.790903,0.788586,0.786733,0.788740,0.001706
